# Initial Setup

In [1]:
# ------------------ Setup ------------------
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import polars as pl
from tqdm.auto import tqdm
import random
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
import kaggle_evaluation.mitsui_inference_server
import joblib
import json
import pickle
import psutil
import shutil

# Make folders
os.makedirs("models", exist_ok=True)

# Global variables
timesteps =5
n_future=4
emw_lookback=26

# ------------------ Data Loading ------------------
train_df = pl.read_csv('/kaggle/input/mitsui-commodity-prediction-challenge/train.csv').to_pandas()
train_labels_df = pl.read_csv('/kaggle/input/mitsui-commodity-prediction-challenge/train_labels.csv').to_pandas()
target_pairs_df = pl.read_csv('/kaggle/input/mitsui-commodity-prediction-challenge/target_pairs.csv').to_pandas()
test_df = pl.read_csv('/kaggle/input/mitsui-commodity-prediction-challenge/test.csv').to_pandas()

# Load pre-calculated features dictionary
with open ('/kaggle/input/mi-rf-feature-selection/featureSetDict_MI_RF.json') as f:
    featureSetDict=json.load(f)    


2025-10-06 12:20:36.150817: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759753236.389756      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759753236.458100      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Log Memory Usage

In [2]:
def log_memory_usage(stage_name=""):
    """
    Logs the current memory usage of the process to the console.
    """
    process = psutil.Process(os.getpid())
    memory_gb = process.memory_info().rss / (1024 ** 3)  # Resident Set Size in GB
    print(f"--- RAM USAGE at '{stage_name}': {memory_gb:.3f} GB ---")

# Return Functions

In [3]:
# ------------------ Utility Functions ------------------
def generate_log_returns_forward(data, lag):
    log_returns = pd.Series(np.nan, index=data.index)
    for t in range(len(data)-lag):
        try:
            log_returns.iloc[t] = np.log(data.iloc[t + 1 + lag ] / data.iloc[t + 1])
        except Exception:
            log_returns.iloc[t] = np.nan
    return log_returns


def generate_targets(column_a: pd.Series, lag: int, column_b: pd.Series = None) -> pd.Series:
    a_returns = generate_log_returns_forward(column_a, lag)
    if column_b is not None:
        b_returns = generate_log_returns_forward(column_b, lag)
        return a_returns - b_returns
    else:
        return a_returns

def generate_log_returns_backward(data, lag):
    log_returns = pd.Series(np.nan, index=data.index)
    for t in range(lag,len(data)):
        try:
            log_returns.iloc[t] = np.log(data.iloc[t] / data.iloc[t-lag])
        except Exception:
            log_returns.iloc[t] = np.nan
    return log_returns

def generate_differences_backward(data, lag):
    differences = pd.Series(np.nan, index=data.index)
    for t in range(lag,len(data)):
        try:
            differences.iloc[t] = data.iloc[t] - data.iloc[t-lag]
        except Exception:
            differences.iloc[t] = np.nan
    return differences

# Seed everything

In [4]:
import random

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(42)

# Data pre-processing

In [5]:
#drop "US_Stock_GOLD* columns as too few data points (and nothing to test)"
train_df.drop(columns=[
'US_Stock_GOLD_adj_open',
'US_Stock_GOLD_adj_high',
'US_Stock_GOLD_adj_low',
'US_Stock_GOLD_adj_close',
'US_Stock_GOLD_adj_volume'], inplace=True)

#fill the gaps on the time series with the most recent data point available, then the soonest
train_df.fillna(method='ffill', inplace=True)
train_df.fillna(method='bfill', inplace=True)
train_df.fillna(0, inplace=True)

test_df.fillna(method='ffill', inplace=True)
test_df.fillna(method='bfill', inplace=True)
train_df.fillna(0, inplace=True)


# Feature Engineering functions

In [6]:
def generate_features_returns (feature_df:pd.DataFrame)->pd.DataFrame:

    log_memory_usage("Before feature engineering")

    # First, downcast the input DataFrame to save memory from the start
    for col in feature_df.select_dtypes(include=['float64']).columns:
        feature_df[col] = feature_df[col].astype('float32')
    for col in feature_df.select_dtypes(include=['int64']).columns:
        feature_df[col] = feature_df[col].astype('int32')

    print("generating feature returns:")
    # ... code for feature returns ...
    
    print("generating feature returns: \n\n")
    
    if 'date_id' in feature_df.columns:
        feature_df.drop(columns=['date_id'], inplace=True)

    if 'is_scored' in feature_df.columns:
        feature_df.drop(columns=['is_scored'], inplace=True)
    
    return_feature_df = pd.DataFrame(index=feature_df.index).astype('float32')
    fx_return_feature_df = pd.DataFrame(index=feature_df.index).astype('float32')
    volume_feature_df = pd.DataFrame(index=feature_df.index).astype('float32')
    oi_feature_df = pd.DataFrame(index=feature_df.index).astype('float32')
    open_price_df = pd.DataFrame(index=feature_df.index).astype('float32')
    close_price_df = pd.DataFrame(index=feature_df.index).astype('float32')
    low_price_df = pd.DataFrame(index=feature_df.index).astype('float32')
    high_price_df = pd.DataFrame(index=feature_df.index).astype('float32')
    volume_df = pd.DataFrame(index=feature_df.index).astype('float32')
    fx_rate_df= pd.DataFrame(index=feature_df.index).astype('float32')

    #calculate 1-day log returns for different lags, fillna
    for col  in tqdm(feature_df, total=len(feature_df.columns)):
        if ('volume' in col) | ('Volume' in col):
            volume_feature_df[col] = feature_df[col]
            volume_feature_df[col].fillna(method='ffill', inplace=True)
            volume_feature_df[col].fillna(method='bfill', inplace=True)

            if 'volume' in col:
                colNew= col.replace('_volume', '')
            elif 'Volume' in col:
                colNew= col.replace('_Volume', '')

            volume_df[colNew] = feature_df[col]
            volume_df[colNew].fillna(method='ffill', inplace=True)
            volume_df[colNew].fillna(method='bfill', inplace=True)

        elif 'open_interest' in col:
            oi_feature_df[col]=generate_differences_backward(feature_df[col], 1)
            oi_feature_df[col].fillna(method='ffill', inplace=True)
            oi_feature_df[col].fillna(method='bfill', inplace=True)

        elif ('close' in col) | ('Close' in col): #price data

            return_feature_df[col] = generate_log_returns_backward(feature_df[col], 1)
            return_feature_df[col].fillna(0, inplace=True)

            if 'close'in col:
              colNew= col.replace('_close', '')
            elif 'Close'in col:
              colNew= col.replace('_Close', '')

            close_price_df[colNew] = feature_df[col]
            close_price_df[colNew].fillna(method='ffill', inplace=True)
            close_price_df[colNew].fillna(method='bfill', inplace=True)

        elif 'FX_' in col: #FX rate data

            fx_return_feature_df[col] = generate_log_returns_backward(feature_df[col], 1)
            fx_return_feature_df[col].fillna(0, inplace=True)

            fx_rate_df[col] = feature_df[col]
            fx_rate_df[col].fillna(method='ffill', inplace=True)
            fx_rate_df[col].fillna(method='bfill', inplace=True)

        elif ('open'in col) | ('Open'in col):

            if 'open'in col:
              colNew= col.replace('_open', '')
            elif 'Open'in col:
              colNew= col.replace('_Open', '')

            open_price_df[colNew] = feature_df[col]
            open_price_df[colNew].fillna(method='ffill', inplace=True)
            open_price_df[colNew].fillna(method='bfill', inplace=True)

        elif ('low'in col) | ('Low'in col):

            if 'low'in col:
              colNew= col.replace('_low', '')
            elif 'Low'in col:
              colNew= col.replace('_Low', '')

            low_price_df[colNew] = feature_df[col]
            low_price_df[colNew].fillna(method='ffill', inplace=True)
            low_price_df[colNew].fillna(method='bfill', inplace=True)

        elif ('high'in col) | ('High'in col):

            if 'high'in col:
              colNew= col.replace('_high', '')
            elif 'High'in col:
              colNew= col.replace('_High', '')

            high_price_df[colNew] = feature_df[col]
            high_price_df[colNew].fillna(method='ffill', inplace=True)
            high_price_df[colNew].fillna(method='bfill', inplace=True)

    # ... code for price structure ...
    log_memory_usage("Before price structure features") # <<< KEY ADDITION
    
    print ("generating price structure and volatility features", "\n")
    
    dailyPriceRange= (high_price_df - low_price_df).astype('float32')
    dailyPriceRange.columns = [col + '_dailyPriceRange' for col in dailyPriceRange.columns]

    intradayReturn= (open_price_df - close_price_df).astype('float32')
    intradayReturn.columns = [col + '_intradayReturn' for col in intradayReturn.columns]

    positionInRange= ((close_price_df - low_price_df) / (high_price_df - low_price_df)).astype('float32')
    positionInRange.columns = [col + '_positionInRange' for col in positionInRange.columns]

    bodyToShadowRatio= (close_price_df - open_price_df) / (high_price_df - low_price_df).astype('float32')
    bodyToShadowRatio.columns = [col + '_bodyToShadowRatio' for col in bodyToShadowRatio.columns]

    log_memory_usage("Before creating momentum indicators") 
    
    print ("creating momentum indicators", '\n')

    momentumIndicators = create_technical_indicator_features(close_price_df, volume_df).astype('float32')
    fx_momentumIndicators = create_technical_indicator_features(fx_rate_df).astype('float32')

    log_memory_usage("After momentum indicators") 
    
    print ("momentum indicators completed", '\n')
    
    # combine features
    #all_feature_df=pd.concat([return_feature_df, fx_return_feature_df, volume_feature_df, oi_feature_df, dailyPriceRange, intradayReturn, positionInRange, bodyToShadowRatio, momentumIndicators, fx_momentumIndicators], axis=1)

   # Start with the first DataFrame as the base
    all_feature_df = return_feature_df.copy()
    del return_feature_df
    gc.collect()
    log_memory_usage("After adding return_feature_df")
    
    # Concatenate and delete one by one
    all_feature_df = pd.concat([all_feature_df, fx_return_feature_df], axis=1)
    del fx_return_feature_df
    gc.collect()
    log_memory_usage("After adding fx_return_feature_df")
    
    all_feature_df = pd.concat([all_feature_df, volume_feature_df], axis=1)
    del volume_feature_df
    gc.collect()
    log_memory_usage("After adding volume_feature_df")
    
    all_feature_df = pd.concat([all_feature_df, oi_feature_df], axis=1)
    del oi_feature_df
    gc.collect()
    log_memory_usage("After adding oi_feature_df")
    
    all_feature_df = pd.concat([all_feature_df, dailyPriceRange], axis=1)
    del dailyPriceRange
    gc.collect()
    log_memory_usage("After adding dailyPriceRange")
    
    all_feature_df = pd.concat([all_feature_df, intradayReturn], axis=1)
    del intradayReturn
    gc.collect()
    log_memory_usage("After adding intradayReturn")
    
    all_feature_df = pd.concat([all_feature_df, positionInRange], axis=1)
    del positionInRange
    gc.collect()
    log_memory_usage("After adding positionInRange")
    
    all_feature_df = pd.concat([all_feature_df, bodyToShadowRatio], axis=1)
    del bodyToShadowRatio
    gc.collect()
    log_memory_usage("After adding bodyToShadowRatio")
    
    all_feature_df = pd.concat([all_feature_df, momentumIndicators], axis=1)
    del momentumIndicators
    gc.collect()
    log_memory_usage("After adding momentumIndicators")
    
    all_feature_df = pd.concat([all_feature_df, fx_momentumIndicators], axis=1)
    del fx_momentumIndicators
    gc.collect()
    log_memory_usage("After final concatenation")
    
    print("All features combined successfully.")

    
    for col in all_feature_df.columns:
        if all_feature_df[col].dtype == 'float64':
            all_feature_df[col] = all_feature_df[col].astype('float32')

    all_feature_df.fillna(method='ffill', inplace=True)
    all_feature_df.fillna(method='bfill', inplace=True)

    #remove columns which have all NaN values
    all_feature_df.dropna(axis=1, how='all', inplace=True)

    return all_feature_df

In [7]:
def create_technical_indicator_features(close_price_df, volume_df=None):
    """
    Engineers momentum features from a DataFrame with 'Close' and 'Volume'.
    """

    """Check if there is volume time series corresponding to the priceSeries

    1. replace "_close" or "_Close" at the end of the priceSeries name with "_volume" , call it volumeName

    2. check if volumeName existis in scaledFeatures columns
    """

    df_macd=pd.DataFrame(index=close_price_df.index)
    df_rsi=pd.DataFrame(index=close_price_df.index)
    df_momentum_x_volume=pd.DataFrame(index=close_price_df.index)

    #for priceSerie in priceSeries:
    # 1. Price Rate of Change (10-day and 21-day momentum)
    momentum_10d = close_price_df.pct_change(periods=10).astype('float32')
    momentum_21d = close_price_df.pct_change(periods=21).astype('float32')

    # 2. MACD
    ema_12 = close_price_df.ewm(span=12, adjust=False).mean().astype('float32')
    ema_26 = close_price_df.ewm(span=emw_lookback, adjust=False).mean().astype('float32')
    macd= ema_12 - ema_26
    df_macd = macd.ewm(span=9, adjust=False).mean().astype('float32')
    df_macd.columns = [col + '_macd' for col in df_macd.columns]

    # 3. Relative Strength Index (RSI)
    delta = close_price_df.diff().astype('float32')
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean().astype('float32')
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean().astype('float32')
    rs = gain / loss
    df_rsi= 100 - (100 / (1 + rs))
    df_rsi.columns = [col + '_rsi' for col in df_rsi.columns]


    # 4. Volume-Adjusted Momentum
    if volume_df is not None:
      volume_ma_20d = volume_df.rolling(window=20).mean().astype('float32')
      df_momentum_x_volume = momentum_10d * (volume_df / volume_ma_20d)
      df_momentum_x_volume.columns = [col + '_momentum_x_volume' for col in df_momentum_x_volume.columns]

    df_techIndicators=pd.concat([df_macd, df_rsi, df_momentum_x_volume], axis=1)

    return df_techIndicators

# Scale features

In [8]:
from sklearn.preprocessing import StandardScaler

def scale_train_features (train_feature_df):

    log_memory_usage("Before scaling")
    scaler = StandardScaler()

    # Fit the scaler ONLY on your training data
    scaler_fit_train = scaler.fit(train_feature_df)

    log_memory_usage("After scaling")
    # Save the scaler to a file
    with open('scaler_fit_train.pkl', 'wb') as f:
        pickle.dump(scaler_fit_train, f)
    
    scaled_train_features = scaler_fit_train.transform(train_feature_df).astype('float32')
    scaled_train_features = pd.DataFrame(scaled_train_features, columns=allFeatureReturns.columns)

    log_memory_usage("After scaling")

    
    return scaler_fit_train, scaled_train_features


# Feature selection MI + Random Forest

In [9]:


# Assume X is your feature DataFrame and y is your target Series

def select_features_MI_randomforest(scaledFeatures, target_pairs_df, train_labels) :
    featureSetDict={}
    for i, row in tqdm(target_pairs_df.iterrows(), total=len(target_pairs_df)):
        target_name = row['target']
        lag = row['lag']
        pair = row['pair']
        target_series = train_labels[target_name]
        target_series.fillna(0, inplace=True)

        log_memory_usage("Calculating MI Scores")
        # --- Step 1: Filter with Mutual Information ---
        mi_scores = mutual_info_regression(scaledFeatures, target_series)
        mi_series = pd.Series(mi_scores, index=allFeatureReturns.columns)
        mi_series = mi_series.sort_values(ascending=False)
        log_memory_usage("After MI Scores")
        
        # Keep features with MI score above a certain threshold or the top 200
        top_mi_features = mi_series.head(300).index
        X_filtered = allFeatureReturns[top_mi_features]

        log_memory_usage("Before Random Forest")
        # --- Step 2: Rank with a Random Forest ---
        rf = RandomForestRegressor(n_estimators=100, random_state=42)
        rf.fit(X_filtered, target_series)
        log_memory_usage("Random Forest fit")

        importance_series = pd.Series(rf.feature_importances_, index=X_filtered.columns)
        importance_series = importance_series.sort_values(ascending=False)

        # Select the final set of top N features
        final_features = importance_series.head(100).index.tolist()
        featureSetDict[target_name]= final_features

    with open('featureSetDict_MI_RF.json', 'w') as f:
        json.dump(featureSetDict, f, indent=4) # indent=4 makes it nicely formatted

    return featureSetDict


In [10]:
def create_multi_step_dataset(feature_series,timesteps, n_future ,target_series=None) -> tuple[np.ndarray, np.ndarray] | np.ndarray:
    """
    Creates a dataset for multi-step forecasting.

    Args:
        feature_series: Series of input features.
        target_series: Series of target values, or None.
        timesteps: The lookback window size.
        n_future: The number of future steps to predict.
    """
    print("generating multi-step dataset: \n\n")

    if target_series is not None:
        target_name = target_series.name
         #find the 'lag' column for the row where 'target_name' == target_name on the target_pairs dataframe
        lag = target_pairs_df[target_pairs_df['target'] == target_name]['lag'].iloc[0]
         #find the 'pair' column for the row where 'target_name' == target_name on the target_pairs dataframe
        pair = target_pairs_df[target_pairs_df['target'] == target_name]['pair'].iloc[0]

        if ' - ' in pair:
          a, b = pair.split(' - ')
          target_series_replicated = generate_targets(train_df[a],1,train_df[b])

        else:
          target_series_replicated=generate_targets(train_df[pair],1)

        target_series_replicated.fillna(0, inplace=True)

    X = []
    y = []

    # Adjust the loop to ensure there's enough data for the future sequence
    for i in range(timesteps, len(feature_series) - n_future):
        # Input sequence (past data)
        X.append(feature_series[i-timesteps:i])
        if target_series is not None:
            # Output sequence (future data)
            y.append(target_series_replicated[i-timesteps : i-timesteps+n_future].to_numpy())

    X_array = np.array(X)

    if target_series is not None:
        y_array = np.array(y)
        return X_array, y_array
    else:
        return X_array

# Execute feature engineering

In [11]:
allFeatureReturns= generate_features_returns(train_df)
scaler_train, scaledFeatures = scale_train_features(allFeatureReturns)

--- RAM USAGE at 'Before feature engineering': 0.918 GB ---
generating feature returns:
generating feature returns: 




100%|██████████| 552/552 [00:13<00:00, 41.55it/s]


--- RAM USAGE at 'Before price structure features': 0.920 GB ---
generating price structure and volatility features 

--- RAM USAGE at 'Before creating momentum indicators': 0.927 GB ---
creating momentum indicators 

--- RAM USAGE at 'After momentum indicators': 0.939 GB ---
momentum indicators completed 

--- RAM USAGE at 'After adding return_feature_df': 0.939 GB ---
--- RAM USAGE at 'After adding fx_return_feature_df': 0.939 GB ---
--- RAM USAGE at 'After adding volume_feature_df': 0.939 GB ---
--- RAM USAGE at 'After adding oi_feature_df': 0.939 GB ---
--- RAM USAGE at 'After adding dailyPriceRange': 0.941 GB ---
--- RAM USAGE at 'After adding intradayReturn': 0.941 GB ---
--- RAM USAGE at 'After adding positionInRange': 0.943 GB ---
--- RAM USAGE at 'After adding bodyToShadowRatio': 0.943 GB ---
--- RAM USAGE at 'After adding momentumIndicators': 0.950 GB ---
--- RAM USAGE at 'After final concatenation': 0.955 GB ---
All features combined successfully.
--- RAM USAGE at 'Before sc

# Create LSTM model

In [12]:
def construct_LSTM_model(timesteps,num_features) -> Sequential:
  # Define the LSTM model
  # Example LSTM model for time series prediction
  model = Sequential()
  model.add(LSTM(units=100, return_sequences=True, input_shape=(timesteps, num_features)))
  model.add(Dropout(0.2))
  model.add(LSTM(units=100))
  model.add(Dropout(0.2))
  model.add(Dense(n_future))  # Prediction of next 4 price movements
  model.compile(optimizer='adam', loss='mean_squared_error')
  return model

# Training Models

In [13]:
# ------------------ Training Models ------------------
def train_LSTM_models(scaledFeatures:pd.DataFrame,featureSetDict, train_labels)-> dict[str, Sequential]:

    models = {}

    # all features  shared across all individual targets (for now)
    for i, row in tqdm(target_pairs_df.iterrows(), total=len(target_pairs_df)):
        target_name = row['target']
        lag = row['lag']
        pair = row['pair']
        target_series = train_labels[target_name]
        target_series.fillna(method='ffill', inplace=True)
        target_series.fillna(0, inplace=True)

        scaledFeatureSubset = scaledFeatures[featureSetDict[target_name]]
    
        X, y = create_multi_step_dataset(scaledFeatureSubset, timesteps, n_future, target_series)
        
        # 2. Build a new model with the correct input shape
        model = construct_LSTM_model(timesteps, X.shape[2])

         # Callbacks can be defined outside the scope
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        # Train the model. The .fit() method will automatically distribute
        # the data batches to the different GPU replicas.
        history = model.fit(
            X, y,
            epochs=200,
            validation_split=0.2,
            verbose=0,
            callbacks=[early_stopping],
            batch_size=64  # It's good practice to use a batch size divisible by the number of GPUs
        )
        
        # 1. Get the total number of epochs that were run
        epochs_trained = len(history.history['val_loss'])

        # 2. Find the epoch with the best validation loss
        # np.argmin finds the index of the minimum value in the list.
        # We add 1 because epochs are 1-based, not 0-based.
        best_epoch = np.argmin(history.history['val_loss']) + 1

        print(f"Training stopped after {epochs_trained} epochs.")
        print(f"The best model was found at epoch {best_epoch} with a validation loss of {min(history.history['val_loss']):.4f}.")
        val_loss = min(history.history['val_loss'])
        print(f"  --> Target: {target_name}, Best Validation Loss: {val_loss:.4f}")
        os.makedirs("models", exist_ok=True)
        model.save(f"models/{target_name}_model.keras")
        models[target_name] = model

    return models

# Execute training

In [14]:
train_LSTM_models(scaledFeatures, featureSetDict,train_labels_df)

  0%|          | 0/424 [00:00<?, ?it/s]

generating multi-step dataset: 




2025-10-06 12:21:08.731274: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
  0%|          | 1/424 [00:25<2:58:11, 25.28s/it]

Training stopped after 44 epochs.
The best model was found at epoch 39 with a validation loss of 0.0001.
  --> Target: target_0, Best Validation Loss: 0.0001
generating multi-step dataset: 




  0%|          | 2/424 [00:46<2:42:05, 23.05s/it]

...

100%|█████████▉| 423/424 [2:01:53<00:16, 16.91s/it]

Training stopped after 30 epochs.
The best model was found at epoch 25 with a validation loss of 0.0003.
  --> Target: target_422, Best Validation Loss: 0.0003
generating multi-step dataset: 




100%|██████████| 424/424 [2:02:09<00:00, 17.29s/it]

Training stopped after 24 epochs.
The best model was found at epoch 19 with a validation loss of 0.0011.
  --> Target: target_423, Best Validation Loss: 0.0011


{'target_0': <Sequential name=sequential, built=True>,
 'target_1': <Sequential name=sequential_1, built=True>,
 'target_2': <Sequential name=sequential_2, built=True>,
 'target_3': <Sequential name=sequential_3, built=True>,
 'target_4': <Sequential name=sequential_4, built=True>,
 'target_5': <Sequential name=sequential_5, built=True>,
 'target_6': <Sequential name=sequential_6, built=True>,
 'target_7': <Sequential name=sequential_7, built=True>,
 'target_8': <Sequential name=sequential_8, built=True>,
 'target_9': <Sequential name=sequential_9, built=True>,
 'target_10': <Sequential name=sequential_10, built=True>,
 'target_11': <Sequential name=sequential_11, built=True>,
 'target_12': <Sequential name=sequential_12, built=True>,
 'target_13': <Sequential name=sequential_13, built=True>,
 'target_14': <Sequential name=sequential_14, built=True>,
 'target_15': <Sequential name=sequential_15, built=True>,
 'target_16': <Sequential name=sequential_16, built=True>,
 'target_17': <Sequ